# <b> NOTE BOOK PROJET DE CLASSIFICATION AUTOMATIQUE DES ENTRPRISES SELON La CODIFICATION NAEMAS </b>

In [1]:

import os
os.chdir('C:/Users/mombengue/Desktop/REP_PYTHON')
os.getcwd()


'C:\\Users\\mombengue\\Desktop\\REP_PYTHON'

# <b> I. PREPARATION DES DONNEES </b>

In [2]:
# Installation du module pandas dans cet environnement
#! pip install pandas

In [3]:
# installation de la  dépendance openpyxl.
#!pip install openpyxl
# installation de nltk
#!pip install nltk

In [16]:
# Chargement de la base
import pandas as pd
base=pd.read_excel("repertoire_entreprise_2022.xlsx", sheet_name=0)
base.shape


(25643, 14)

In [17]:
#base.head(10)

In [6]:
base.shape

(25643, 14)

In [7]:
# Selection des variables en jeu
df=base[["DESIGNATION_PRECISE_ACTIVITE","RAISON_SOCIALE","SYSCOA1","NAEMAS"]]
df.head(10)

,DESIGNATION_PRECISE_ACTIVITE,RAISON_SOCIALE,SYSCOA1,NAEMAS
0,BOULANGERIE,BOULANGERIE JALAL (MARWAN ABDEL AL),11001,F00060
1,COMMERCE GENERAL,GIE PADS,31003,N00030
2,VENTE DE CARTES POSTALES (EX-EDITION),AFRICA INFORMATION DE L'ELITE SA,31003,Q00050
3,SERVICE,MAKE SUARL,38003,N00020
4,OPTICIEN,FILAO'PTIC,26002,J00140
5,LOCATION DE BIENS IMMOBILIERS,SCI KLEBER,37002,S00000
6,GARDIENNAGE,AGENCE DE SURVEILLANCE ET DE GARDIENAGE - DANI...,38003,U00030
7,BTP,SPIE CAPAG SA (SUCCURSALE),30001,M00020
8,NaN,LA BAGUETTE (MOHAMED KASSEM OMAIS),29004,K00010
9,ACTIVITES POUR LA SANTE DES HOMMES (OPHTALMOLO...,CABINET OPHTALMOLOGIQUE DR VICTOR AMOUSSOU,41001,X00010


In [8]:
# Détection des lignes manquantes 
df.isnull().sum() 

DESIGNATION_PRECISE_ACTIVITE    856
RAISON_SOCIALE                    0
SYSCOA1                           0
NAEMAS                            5
dtype: int64

In [9]:
# Suppression des lignes manquantes
df= df.dropna()

# Vérification 
df.isnull().sum()

DESIGNATION_PRECISE_ACTIVITE    0
RAISON_SOCIALE                  0
SYSCOA1                         0
NAEMAS                          0
dtype: int64

In [10]:
df.shape

(24782, 4)

In [18]:
# effectif des classes (Les classes sont trop déséquilibrées)
#df["NAEMAS"].value_counts()

In [12]:
# On Mélange le jeu de données

In [13]:
from sklearn.utils import shuffle

# Utiliser la fonction shuffle() pour mélanger les lignes du DataFrame
df = shuffle(df, random_state=42)

# Afficher le DataFrame mélangé
df.head(10)

,DESIGNATION_PRECISE_ACTIVITE,RAISON_SOCIALE,SYSCOA1,NAEMAS
3300,BAR - RESTAURANT,LE TACOMA - SARL,33002,P00020
4433,ACTIVITES POUR LA SANTE DES HOMMES,CRE (CABINET DE RADIOLOGIE ET D'ECHOGRAPHIE),41001,X00010
13808,COMMERCE,ETS AMADOU DIONGUE FALL,31003,N00030
3256,BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,IGUANE CAFE - CUBAN BAR (CHRISTIAN J. E. BRIAN...,33002,P00020
7879,NETTOYAGE GENERAL PROFESSIONNEL,ENTREPRISE SERIGNE FALLOU SUARL,38003,U00040
7004,PEINTRE - ETANCHEITE,EPE (ENTREPRISE DE PEINTURE ET D'ETACHEITE) JE...,30002,M00030
24394,CONSEIL EN INFORMATIQUE,3TI RESEARCH SENEGAL - SARL,38002,Q00050
10614,MENUISERIE METALLIQUE,ENTREPRISE GENERAL DIALLO ET FRERES,24002,N00030
7066,VENTES DE VEHICULES,TATA AFRICA SENEGAL SARL,31001,N00010
23788,RESTAURATION,LE GLACIER DE LA PETITE COETE SUARL,33002,P00020


# <b> I. TECHNIQUES CLASSIQUES </b>

## <b> II.1 Jeu de données avec "DESIGNATION_PRECISE_ACTIVITE" comme variable explicative </b>

In [14]:
# Selection des variables en jeu
df1=df[["DESIGNATION_PRECISE_ACTIVITE","NAEMAS"]]
df1.head(10)

,DESIGNATION_PRECISE_ACTIVITE,NAEMAS
3300,BAR - RESTAURANT,P00020
4433,ACTIVITES POUR LA SANTE DES HOMMES,X00010
13808,COMMERCE,N00030
3256,BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,P00020
7879,NETTOYAGE GENERAL PROFESSIONNEL,U00040
7004,PEINTRE - ETANCHEITE,M00030
24394,CONSEIL EN INFORMATIQUE,Q00050
10614,MENUISERIE METALLIQUE,N00030
7066,VENTES DE VEHICULES,N00010
23788,RESTAURATION,P00020


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# vectorisation des descriptions
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df1['DESIGNATION_PRECISE_ACTIVITE'])

y = df['NAEMAS']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Liste des modèles
models = {
    'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs'),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': MultinomialNB()
}

# Dictionnaire pour stocker les résultats
results = {}

for model_name, model in models.items():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    # Stockage des résultats
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
          }
# Affichage des résultats
results_df = pd.DataFrame(results).T
print(results_df)

C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


                        Accuracy  Precision    Recall  F1 Score
Logistic Regression     0.680452   0.689851  0.680452  0.650175
Support Vector Machine  0.682066   0.689243  0.682066  0.656035
Random Forest           0.676619   0.662775  0.676619  0.657437
Naive Bayes             0.623563   0.669032  0.623563  0.558086


### <b> II.2 Jeu de données avec DESIGNATION_PRECISE_ACTIVITE et RAISON_SOCIALE comme variables explicatives </b>

In [39]:
# Selection des variables en jeu
df2=df[["DESIGNATION_PRECISE_ACTIVITE","RAISON_SOCIALE","NAEMAS"]]
df2.head(10)

,DESIGNATION_PRECISE_ACTIVITE,RAISON_SOCIALE,NAEMAS
3300,BAR - RESTAURANT,LE TACOMA - SARL,P00020
4433,ACTIVITES POUR LA SANTE DES HOMMES,CRE (CABINET DE RADIOLOGIE ET D'ECHOGRAPHIE),X00010
13808,COMMERCE,ETS AMADOU DIONGUE FALL,N00030
3256,BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,IGUANE CAFE - CUBAN BAR (CHRISTIAN J. E. BRIAN...,P00020
7879,NETTOYAGE GENERAL PROFESSIONNEL,ENTREPRISE SERIGNE FALLOU SUARL,U00040
7004,PEINTRE - ETANCHEITE,EPE (ENTREPRISE DE PEINTURE ET D'ETACHEITE) JE...,M00030
24394,CONSEIL EN INFORMATIQUE,3TI RESEARCH SENEGAL - SARL,Q00050
10614,MENUISERIE METALLIQUE,ENTREPRISE GENERAL DIALLO ET FRERES,N00030
7066,VENTES DE VEHICULES,TATA AFRICA SENEGAL SARL,N00010
23788,RESTAURATION,LE GLACIER DE LA PETITE COETE SUARL,P00020


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Prétraitement des colonnes de texte en utilisant TF-IDF
vectorizer1 = TfidfVectorizer()
X_texte1 = vectorizer1.fit_transform(df2['DESIGNATION_PRECISE_ACTIVITE'])

vectorizer2 = TfidfVectorizer()
X_texte2 = vectorizer2.fit_transform(df2['RAISON_SOCIALE'])

# Prétraitement de la colonne numérique
#scaler = StandardScaler()
#X_numerique = df1['SYSCOA1']

# Combinaison des caractéristiques traitées
from scipy.sparse import hstack
X = hstack([X_texte1, X_texte2])
# Séparation des caractéristiques et de la cible
y = df2['NAEMAS']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Liste des modèles
models = {
    'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs'),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': MultinomialNB()
    }

# Dictionnaire pour stocker les résultats
results = {}

for model_name, model in models.items():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    # Stockage des résultats
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Affichage des résultats
results_df = pd.DataFrame(results).T
print(results_df)


C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                        Accuracy  Precision    Recall  F1 Score
Logistic Regression     0.687714   0.693544  0.687714  0.659643
Support Vector Machine  0.674803   0.693514  0.674803  0.639782
Random Forest           0.687916   0.670545  0.687916  0.655591
Naive Bayes             0.588662   0.710284  0.588662  0.516820


### <b> II.3 Jeu de données avec DESIGNATION_PRECISE_ACTIVITE, RAISON_SOCIALE et SYSCOA1 comme variables explicatives </b>

In [41]:
df3=df[["DESIGNATION_PRECISE_ACTIVITE","RAISON_SOCIALE","SYSCOA1","NAEMAS"]]
df3.head(10)

,DESIGNATION_PRECISE_ACTIVITE,RAISON_SOCIALE,SYSCOA1,NAEMAS
3300,BAR - RESTAURANT,LE TACOMA - SARL,33002,P00020
4433,ACTIVITES POUR LA SANTE DES HOMMES,CRE (CABINET DE RADIOLOGIE ET D'ECHOGRAPHIE),41001,X00010
13808,COMMERCE,ETS AMADOU DIONGUE FALL,31003,N00030
3256,BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,IGUANE CAFE - CUBAN BAR (CHRISTIAN J. E. BRIAN...,33002,P00020
7879,NETTOYAGE GENERAL PROFESSIONNEL,ENTREPRISE SERIGNE FALLOU SUARL,38003,U00040
7004,PEINTRE - ETANCHEITE,EPE (ENTREPRISE DE PEINTURE ET D'ETACHEITE) JE...,30002,M00030
24394,CONSEIL EN INFORMATIQUE,3TI RESEARCH SENEGAL - SARL,38002,Q00050
10614,MENUISERIE METALLIQUE,ENTREPRISE GENERAL DIALLO ET FRERES,24002,N00030
7066,VENTES DE VEHICULES,TATA AFRICA SENEGAL SARL,31001,N00010
23788,RESTAURATION,LE GLACIER DE LA PETITE COETE SUARL,33002,P00020


In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Prétraitement des colonnes de texte en utilisant TF-IDF
vectorizer1 = TfidfVectorizer()
X_texte1 = vectorizer1.fit_transform(df3['DESIGNATION_PRECISE_ACTIVITE'])

vectorizer2 = TfidfVectorizer()
X_texte2 = vectorizer2.fit_transform(df3['RAISON_SOCIALE'])

# Conversion de "SYSCOA1" en chaîne de caractères et transformation TF-IDF
vectorizer3 = TfidfVectorizer()
X_texte3 = vectorizer3.fit_transform(df3['SYSCOA1'].astype('str'))

# Combinaison des caractéristiques traitées

X = hstack([X_texte1, X_texte2,X_texte3])

# Séparation des caractéristiques et de la cible
y = df3['NAEMAS']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Liste des modèles
models = {
    'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs'),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': MultinomialNB()
    }

# Dictionnaire pour stocker les résultats
results = {}

for model_name, model in models.items():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    # Stockage des résultats
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Affichage des résultats
results_df = pd.DataFrame(results).T
print(results_df)



C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                        Accuracy  Precision    Recall  F1 Score
Logistic Regression     0.741981   0.735427  0.741981  0.723665
Support Vector Machine  0.737543   0.737745  0.737543  0.714522
Random Forest           0.735324   0.722491  0.735324  0.712638
Naive Bayes             0.680654   0.706115  0.680654  0.626532


# <b> II. METHODE DE COMPARAISON DES VECTEURS DE MOTS </b>

In [43]:
import nltk
from nltk.corpus import stopwords

# Télécharger les stop words de NLTK (à faire une seule fois)
nltk.download('stopwords')

# Récupérer la liste des stop words en français
stop_words = set(stopwords.words('french'))

# Fonction pour filtrer les stop words dans une chaîne de caractères

def remove_stopwords(text, stop_words):
    
    words = text.split()  # Tokenisation
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)  # Reconstituer la chaîne de caractères

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mombengue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
# Vérification de la fonction remove_stopwords(text, stop_words)

text1="Je suis un Sénégalais gentil, humble et avec beaucoup de caractère."

remove_stopwords(text1, stop_words)

'Sénégalais gentil, humble beaucoup caractère.'

In [11]:
# installation de matplotlib pour cet environnement
#!pip install matplotlib

In [12]:
# installation de wordcloud dans l'environnement
#!pip install wordcloud

In [50]:
df4=df[["DESIGNATION_PRECISE_ACTIVITE","SYSCOA1","NAEMAS"]]
df4[df4["NAEMAS"] == 'Q00050']

,DESIGNATION_PRECISE_ACTIVITE,SYSCOA1,NAEMAS
24394,CONSEIL EN INFORMATIQUE,38002,Q00050
15706,SERVICES INFORMATIQUES,38002,Q00050
17052,PROGRAMMATION INFORMATIQUE,38002,Q00050
8374,SERVICES INFORMATIQUES ET DE TRAITEMENT DE DON...,38002,Q00050
11729,SECURITE ELECTRONIQUE,38002,Q00050
...,...,...,...
20821,COMMERCE ALIMENTS VOLAIL ET BETAIL,31003,Q00050
20397,CONSEILS ET AUTRES ACTIVITES INFORMATIQUES,38002,Q00050
25092,PROGRAMMATION INFORMATIQUE,38002,Q00050
15094,NETTOYAGE INDUSTRIEL,38003,Q00050


In [19]:

# Récupération du code 'NAEMAS' sous forme de liste
Code = list(df4['NAEMAS'].unique())

# Initialiser un dictionnaire vide pour stocker les résultats
Dicto_ = {}

# Regroupement des activités ayant le même code NAEMAS
for k in Code:
     desc = df4[df4['NAEMAS'] == k]['DESIGNATION_PRECISE_ACTIVITE'].tolist()

    # Appliquer la fonction remove_stopwords() à chaque chaîne de caractères pour chaque liste du dictionnaire

     filtered_list_of_strings = [remove_stopwords(sentence, stop_words) for sentence in desc]
    
    # Associer la liste filtered_list_of_strings au clé correspondant dans le dictionnaire
     Dicto_[k] = filtered_list_of_strings 

# Afficher le résulta
#print(Dicto_)

NameError: name 'df4' is not defined

# Vecteurs redondants

In [52]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Liste des scores

Scores = []

# Liste des vecteurs
vecteurs = []

# Remplir la liste des vecteurs avec les activités combinées
for k, activities in Dicto_.items():
    vect_k = ' '.join(activities)
    vecteurs.append(vect_k)

# Utiliser le TfidfVectorizer une seule fois pour tous les textes
vectorizer = TfidfVectorizer()

# Pour chaque désignation dans le DataFrame
for row in df4['DESIGNATION_PRECISE_ACTIVITE']:
    # Convertir les vecteurs en une matrice TF-IDF
    vecteurs_matrice = vectorizer.fit_transform(vecteurs + [row])
    
    # Calculer la similarité cosinus
    similarites = cosine_similarity(vecteurs_matrice)
    
    # Extraire les similarités du vecteur à comparer
    similarites_vecteur_a_comparer = similarites[-1][:-1]
    
    # Stocker les scores dans la liste
    Scores.append(similarites_vecteur_a_comparer)

# Convertir la liste des scores en DataFrame
similarity_df = pd.DataFrame(Scores, index=df4['DESIGNATION_PRECISE_ACTIVITE'])

# Utiliser les codes NAEMAS comme noms de colonnes
similarity_df.columns = [f'score_{code}' for code in Dicto_.keys()]

# Afficher le DataFrame de similarité
similarity_df.head(10)


,score_P00020,score_X00010,score_N00030,score_U00040,score_M00030,score_Q00050,score_N00010,score_T00030,score_P00010,score_O00050,...,score_E00010,score_Z00000,score_F00010,score_A00020,score_k00010,score_J00180,score_A00050,score_C00020,score_J00010,score_B00040
DESIGNATION_PRECISE_ACTIVITE,,,,,,,,,,,,,,,,,,,,,
BAR - RESTAURANT,0.724888,0.000000,0.001326,0.000000,0.000000,0.000000,0.000000,0.000000,0.234588,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
ACTIVITES POUR LA SANTE DES HOMMES,0.000279,0.409609,0.003580,0.022827,0.011426,0.035151,0.001093,0.024169,0.005305,0.008176,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
COMMERCE,0.013131,0.005000,0.631783,0.068020,0.036956,0.043159,0.302889,0.042475,0.006929,0.023295,...,0.0,0.130177,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,0.237453,0.000000,0.000700,0.001495,0.000000,0.000000,0.000000,0.000000,0.113922,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
NETTOYAGE GENERAL PROFESSIONNEL,0.003625,0.002128,0.102825,0.060735,0.011158,0.005104,0.024844,0.024177,0.000861,0.002170,...,0.0,0.256252,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
PEINTRE - ETANCHEITE,0.000000,0.000000,0.000000,0.000000,0.017344,0.000000,0.006135,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
CONSEIL EN INFORMATIQUE,0.001202,0.000856,0.009745,0.022460,0.000000,0.236728,0.002443,0.121046,0.000000,0.003738,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
MENUISERIE METALLIQUE,0.000000,0.000000,0.001822,0.002214,0.028089,0.000000,0.004530,0.001248,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
VENTES DE VEHICULES,0.000000,0.000608,0.008128,0.002333,0.000000,0.004198,0.230084,0.000478,0.002106,0.005694,...,0.0,0.000000,0.042061,0.0,0.0,0.162927,0.0,0.0,0.0,0.0


In [53]:
# Ajouter les colonnes SYSCOA1 et NAEMAS
similarity_df['SYSCOA1'] = df4['SYSCOA1'].values
similarity_df['NAEMAS'] = df4['NAEMAS'].values
# Transformer la colonne SYSCOA1 en catégorie
similarity_df['SYSCOA1'] = similarity_df['SYSCOA1'].astype('category')
# Afficher le DataFrame de similarité avec les colonnes ajoutées
similarity_df

,score_P00020,score_X00010,score_N00030,score_U00040,score_M00030,score_Q00050,score_N00010,score_T00030,score_P00010,score_O00050,...,score_F00010,score_A00020,score_k00010,score_J00180,score_A00050,score_C00020,score_J00010,score_B00040,SYSCOA1,NAEMAS
DESIGNATION_PRECISE_ACTIVITE,,,,,,,,,,,,,,,,,,,,,
BAR - RESTAURANT,0.724888,0.000000,0.001326,0.000000,0.000000,0.000000,0.000000,0.000000,0.234588,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,33002,P00020
ACTIVITES POUR LA SANTE DES HOMMES,0.000279,0.409609,0.003580,0.022827,0.011426,0.035151,0.001093,0.024169,0.005305,0.008176,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,41001,X00010
COMMERCE,0.013131,0.005000,0.631783,0.068020,0.036956,0.043159,0.302889,0.042475,0.006929,0.023295,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31003,N00030
BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,0.237453,0.000000,0.000700,0.001495,0.000000,0.000000,0.000000,0.000000,0.113922,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,33002,P00020
NETTOYAGE GENERAL PROFESSIONNEL,0.003625,0.002128,0.102825,0.060735,0.011158,0.005104,0.024844,0.024177,0.000861,0.002170,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,38003,U00040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COMMERCE,0.013131,0.005000,0.631783,0.068020,0.036956,0.043159,0.302889,0.042475,0.006929,0.023295,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31003,N00030
ACTIVITES POUR LA SANTE SANTE DES HOMMES (CONSULTANCE EN BIOLOGIE),0.000211,0.454814,0.003483,0.030091,0.012509,0.034401,0.000825,0.056412,0.004005,0.007447,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,41001,X00010
PHARMACIE,0.000000,0.006673,0.058585,0.003525,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31003,N00030


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Séparation des caractéristiques et de la cible
X = similarity_df.iloc[:, :-1]
y = similarity_df['NAEMAS']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation des données (optionnelle pour Naive Bayes, mais nécessaire pour d'autres modèles)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Liste des modèles
models = {
    'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    #'Naive Bayes': MultinomialNB()
}

# Dictionnaire pour stocker les résultats
results = {}

for model_name, model in models.items():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    # Stockage des résultats
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Affichage des résultats
results_df = pd.DataFrame(results).T
results_df

C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,Accuracy,Precision,Recall,F1 Score
Logistic Regression,0.697398,0.692483,0.697398,0.677958
Support Vector Machine,0.685092,0.675754,0.685092,0.660046
Random Forest,0.700424,0.690809,0.700424,0.683600


# Vecteurs purs

In [20]:
from collections import Counter

# Fusionner toutes les listes pour compter les occurrences des mots
tous_les_mots = [mot for mots in  Dicto_.values() for mot in mots]
compte_mots = Counter(tous_les_mots)

# Identifier le mot le plus fréquent dans chaque liste
mots_a_conserver = {}
for key, liste in  Dicto_.items():
    compteur = Counter(liste)
    mot_plus_frequent = max(compteur, key=lambda mot:(compteur[mot], compte_mots[mot]))
    mots_a_conserver[mot_plus_frequent] = key

# Supprimer les mots des autres listes s'ils sont plus fréquents ailleurs
for mot, liste_a_conserver_key in mots_a_conserver.items():
    for key, liste in  Dicto_.items():
        if key != liste_a_conserver_key and mot in liste:
            liste.remove(mot)

print("Dictionnaire modifié :")
for key, liste in  Dicto_.items():
   # print(f"{key} :", liste)

SyntaxError: incomplete input (1003319446.py, line 22)

# NUAGE DES MOTS

In [21]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Parcourir chaque clé et générer un nuage de mots pour chaque groupe

for key, activities in Dicto_.items():
    
    # Combiner les activités en une seule chaîne de texte
    text = ' '.join(activities)
    
    # Créer un objet WordCloud
    
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    # Afficher le nuage de mots
    
   # plt.figure(figsize=(10, 5))
    #plt.imshow(wordcloud, interpolation='bilinear')
    #plt.title(f'Nuage de mots pour la clé {key}', pad=30)
    #plt.axis('off')
    #plt.show()

NameError: name 'Dicto_' is not defined

In [62]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Dictionnaire des scores
# Dictionnaire des scores
Scores = []

# Liste des vecteurs
vecteurs = []

# Remplir la liste des vecteurs avec les activités combinées
for k, activities in Dicto_.items():
    vect_k = ' '.join(activities)
    vecteurs.append(vect_k)

# Utiliser le TfidfVectorizer une seule fois pour tous les textes
vectorizer = TfidfVectorizer()

# Pour chaque désignation dans le DataFrame
for row in df3['DESIGNATION_PRECISE_ACTIVITE']:
    # Convertir les vecteurs en une matrice TF-IDF
    vecteurs_matrice = vectorizer.fit_transform(vecteurs + [row])
    
    # Calculer la similarité cosinus
    similarites = cosine_similarity(vecteurs_matrice)
    
    # Extraire les similarités du vecteur à comparer
    similarites_vecteur_a_comparer = similarites[-1][:-1]
    
    # Stocker les scores dans la liste
    Scores.append(similarites_vecteur_a_comparer)

# Convertir la liste des scores en DataFrame
similarity_df = pd.DataFrame(Scores, index=df3['DESIGNATION_PRECISE_ACTIVITE'])

# Utiliser les codes NAEMAS comme noms de colonnes
similarity_df.columns = [f'score_{code}' for code in Dicto_.keys()]

# Afficher le DataFrame de similarité
similarity_df.head(10)


,score_P00020,score_X00010,score_N00030,score_U00040,score_M00030,score_Q00050,score_N00010,score_T00030,score_P00010,score_O00050,...,score_E00010,score_Z00000,score_F00010,score_A00020,score_k00010,score_J00180,score_A00050,score_C00020,score_J00010,score_B00040
DESIGNATION_PRECISE_ACTIVITE,,,,,,,,,,,,,,,,,,,,,
BAR - RESTAURANT,0.719040,0.000000,0.001288,0.000000,0.000000,0.000000,0.000000,0.000000,0.225584,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
ACTIVITES POUR LA SANTE DES HOMMES,0.000273,0.436111,0.002982,0.021322,0.010256,0.033261,0.001073,0.023919,0.005030,0.007914,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
COMMERCE,0.010806,0.004030,0.654085,0.067281,0.031877,0.040632,0.315695,0.042899,0.003553,0.020622,...,0.0,0.137868,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,0.235599,0.000000,0.000680,0.001414,0.000000,0.000000,0.000000,0.000000,0.109557,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
NETTOYAGE GENERAL PROFESSIONNEL,0.003597,0.002010,0.099840,0.057452,0.010667,0.004898,0.024747,0.023780,0.000828,0.002059,...,0.0,0.254487,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
PEINTRE - ETANCHEITE,0.000000,0.000000,0.000000,0.000000,0.016580,0.000000,0.006111,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
CONSEIL EN INFORMATIQUE,0.001193,0.000809,0.009463,0.021246,0.000000,0.227092,0.002434,0.119055,0.000000,0.003547,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
MENUISERIE METALLIQUE,0.000000,0.000000,0.000280,0.002187,0.027528,0.000000,0.004444,0.001209,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
VENTES DE VEHICULES,0.000000,0.000574,0.007892,0.002207,0.000000,0.004028,0.229181,0.000470,0.002026,0.005403,...,0.0,0.000000,0.042051,0.0,0.0,0.162927,0.0,0.0,0.0,0.0


In [63]:
# Ajouter les colonnes SYSCOA1 et NAEMAS
similarity_df['SYSCOA1'] = df3['SYSCOA1'].values
similarity_df['NAEMAS'] = df3['NAEMAS'].values
# Transformer la colonne SYSCOA1 en catégorie
similarity_df['SYSCOA1'] = similarity_df['SYSCOA1'].astype('category')
# Afficher le DataFrame de similarité avec les colonnes ajoutées
similarity_df

,score_P00020,score_X00010,score_N00030,score_U00040,score_M00030,score_Q00050,score_N00010,score_T00030,score_P00010,score_O00050,...,score_F00010,score_A00020,score_k00010,score_J00180,score_A00050,score_C00020,score_J00010,score_B00040,SYSCOA1,NAEMAS
DESIGNATION_PRECISE_ACTIVITE,,,,,,,,,,,,,,,,,,,,,
BAR - RESTAURANT,0.719040,0.000000,0.001288,0.000000,0.000000,0.000000,0.000000,0.000000,0.225584,0.000000,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,33002,P00020
ACTIVITES POUR LA SANTE DES HOMMES,0.000273,0.436111,0.002982,0.021322,0.010256,0.033261,0.001073,0.023919,0.005030,0.007914,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,41001,X00010
COMMERCE,0.010806,0.004030,0.654085,0.067281,0.031877,0.040632,0.315695,0.042899,0.003553,0.020622,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31003,N00030
BAR - DISCOTHEQUE - ANIMATION - CUBAN BAR,0.235599,0.000000,0.000680,0.001414,0.000000,0.000000,0.000000,0.000000,0.109557,0.000000,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,33002,P00020
NETTOYAGE GENERAL PROFESSIONNEL,0.003597,0.002010,0.099840,0.057452,0.010667,0.004898,0.024747,0.023780,0.000828,0.002059,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,38003,U00040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COMMERCE,0.010806,0.004030,0.654085,0.067281,0.031877,0.040632,0.315695,0.042899,0.003553,0.020622,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31003,N00030
ACTIVITES POUR LA SANTE SANTE DES HOMMES (CONSULTANCE EN BIOLOGIE),0.000206,0.475748,0.002895,0.028160,0.011404,0.032615,0.000810,0.055296,0.003799,0.007217,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,41001,X00010
PHARMACIE,0.000000,0.006304,0.056887,0.003334,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31003,N00030


In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Séparation des caractéristiques et de la cible
X = similarity_df.iloc[:, :-1]
y = similarity_df['NAEMAS']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation des données (optionnelle pour Naive Bayes, mais nécessaire pour d'autres modèles)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Liste des modèles
models = {
    'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    #'Naive Bayes': MultinomialNB()
}

# Dictionnaire pour stocker les résultats
results = {}

for model_name, model in models.items():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    # Stockage des résultats
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Affichage des résultats
results_df = pd.DataFrame(results).T
print(results_df)



C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


                        Accuracy  Precision    Recall  F1 Score
Logistic Regression     0.696389   0.692708  0.696389  0.676503
Support Vector Machine  0.684083   0.673474  0.684083  0.659270
Random Forest           0.698810   0.686987  0.698810  0.682475


- Se limiter au 3 premier chiffre du code NAMEAS pour faire un regroupement
- voir la possibilité de regrouper certaines classes minoritaires
- Optimisation des modéles avec des hyparamétres
- Utiliser les labels du sycoa à la place des codes
- Transformer les codes syscoa en catégories
- utliser les transformateurs en deep learning

In [11]:
#! pip install scikit-learn

Explication du Code
CountVectorizer : Il convertit les textes en une matrice de fréquence de mots. Chaque mot dans le texte devient une caractéristique dans la matrice.

cosine_similarity : Cette fonction calcule la similarité cosinus entre les vecteurs de caractéristiques. La similarité cosinus est une mesure de similarité qui calcule le cosinus de l'angle entre deux vecteurs dans un espace multidimensionnel. C'est une mesure couramment utilisée pour comparer des documents textuels.

Matrice de Similarité : cosine_similarity retourne une matrice où chaque entrée [i, j] représente la similarité entre le i-ème et le j-ème vecteur.

Extraction des Similarités : La dernière ligne de la matrice contient les similarités entre le vecteur à comparer (vecteur_4) et tous les autres vecteurs.

Utilisation Avancée
Pour des analyses plus sophistiquées, vous pouvez également envisager d'autres techniques de vectorisation et de mesures de similarité, telles que :

Word2Vec : Pour obtenir des vecteurs de mots basés sur des modèles de langage plus complexes.
Doc2Vec : Pour représenter des documents (ou des nuages de mots) en tant que vecteurs.
BERT : Pour des représentations contextuelles des mots et des phrases.
Ces techniques peuvent capturer des nuances plus subtiles dans les textes et améliorer la précision de vos comparaisons. Voici un exemple utilisant TfidfVectorizer :

Le TfidfVectorizer est une classe de la bibliothèque scikit-learn en Python qui permet de convertir une collection de documents de texte brut en une matrice de caractéristiques TF-IDF (Term Frequency-Inverse Document Frequency).
Le TF-IDF est une technique utilisée pour évaluer l'importance d'un mot dans un document par rapport à un corpus de documents.

Comment Fonctionne le TF-IDF

Term Frequency (TF) : Mesure la fréquence d'un terme dans un document. Elle est souvent normalisée pour éviter un biais en faveur de longs documents.
​
Inverse Document Frequency (IDF) : Mesure l'importance d'un terme dans le corpus entier. Elle diminue le poids des termes très fréquents dans le corpus et augmente celui des termes rares.

TF-IDF : Combinaison des deux mesures, calculée comme le produit de TF et IDF :

TF-IDF(t,d,D)=TF(t,d)×IDF(t,D)

Utilisation de TfidfVectorizer en Python
Voici un exemple de code complet utilisant TfidfVectorizer pour transformer des textes en vecteurs TF-IDF et comparer leur similarité :

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Chargement des données
#df = pd.read_excel("repertoire_entreprise_2022.xlsx", sheet_name=0)  # Assurez-vous que le fichier CSV contient les colonnes appropriées

# Prétraitement du texte
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les ponctuations et autres caractères spéciaux
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    return text

df['DESIGNATION_PRECISE_ACTIVITE'] = df['DESIGNATION_PRECISE_ACTIVITE'].apply(preprocess_text)

# Vectorisation du texte
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['DESIGNATION_PRECISE_ACTIVITE'])


# Mélange des lignes du DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Cible
y = df['NAEMAS']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle
model = MultinomialNB()
model.fit(X_train, y_train)

# Prédiction et évaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Prédiction pour une nouvelle description
#new_description = "Description de l'activité de la nouvelle entreprise"
#ew_description_preprocessed = preprocess_text(new_description)
#new_description_vectorized = vectorizer.transform([new_description_preprocessed])
#predicted_code_naemas = model.predict(new_description_vectorized)
#print("Le code NAEMAS prédit pour la nouvelle entreprise est :", predicted_code_naemas[0])


              precision    recall  f1-score   support

      A00010       0.00      0.00      0.00         9
      A00020       0.00      0.00      0.00         1
      A00030       0.00      0.00      0.00         9
      A00040       0.00      0.00      0.00         9
      A00050       0.00      0.00      0.00         1
      A00070       0.00      0.00      0.00        19
      B00010       0.00      0.00      0.00         2
      B00020       0.00      0.00      0.00        13
      B00040       0.00      0.00      0.00         1
      D00010       0.00      0.00      0.00        20
      D00020       0.00      0.00      0.00         2
      E00010       0.00      0.00      0.00         2
      E00020       0.00      0.00      0.00         4
      E00030       0.00      0.00      0.00         3
      E00040       0.00      0.00      0.00         7
      E00050       0.00      0.00      0.00         8
      F00020       0.00      0.00      0.00        26
      F00030       0.00    

C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mombengue\AppData\Local\OneDrive\Mon_Anaconda\envs\Class_NAEMAS\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [ ]:
vecteurs = []
for k, activities in Dicto.items():
    
    # Combiner les activités en une seule chaîne de texte
     vect_k= ' '.join(activities)
     vecteurs.append(vect_k)

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

for row in df[DESIGNATION_PRECISE_ACTIVITE]:
      vecteur_a_comparer = "PRESTATIONS DE SERVICES ( 63%)- VENTE DE MARCHANDISES (37%) -PRODUITS DIVERS"
# Utiliser le TfidfVectorizer
     vectorizer = TfidfVectorizer()

# Convertir les vecteurs en une matrice TF-IDF
     vecteurs_matrice = vectorizer.fit_transform(vecteurs + [vecteur_a_comparer])

# Calculer la similarité cosinus
    similarites = cosine_similarity(vecteurs_matrice)

# Extraire les similarités du vecteur à comparer
similarites_vecteur_a_comparer = similarites[-1][:-1]

# Afficher les résultats
for i, similarite in enumerate(similarites_vecteur_a_comparer):
      print(f"Similarité entre vecteur4 et vecteur_{i+1}: {similarite:.4f}")